In [31]:
import pandas as pd  
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)

In [32]:
result = pd.read_csv('exp3/exp2.5/beer_imputed_epoches-1.csv')
result.drop(['Unnamed: 0'], axis=1, inplace=True)
result['id'] = result['id'].astype('int')
result['brewery_id'] = result['brewery_id'].astype('int')
result['index'] = result['index'].astype('int')
result['ibu'] = result['ibu'].astype('int')
result.set_index('index',inplace=True)
result.head()
# result.loc[5,'id']

,id,beer_name,style,ounces,abv,ibu,brewery_id,brewery_name,city,state
index,,,,,,,,,,
1,1436,Pub Beer,American Pale Lager,12.0 oz,0.05,12,408,10 Barrel Brewing Company,Bend,OR
2,2265,Devil's Cup,American Pale Ale (APA),12.0 oz.,0.066,12,177,18th Street Brewery,Gary,IN
3,2264,Rise of the Phoenix,American IPA,12.0 ounce,0.071,12,177,18th Street Brewery,Gary,IN
4,2263,Sinister,American Double / Imperial IPA,12.0 oz,0.09%,12,177,18th Street Brewery,Gary,IN
6,2261,Black Exodus,Oatmeal Stout,12.0 oz,0.077,16,177,18th Street Brewery,Gary,IN


In [33]:
data = pd.read_csv('raha/datasets/beers/dirty.csv')
data.set_index('index',inplace=True)
data.head()

,id,beer_name,style,ounces,abv,ibu,brewery_id,brewery_name,city,state
index,,,,,,,,,,
1,1436,Pub Beer,American Pale Lager,12.0 oz,0.05,NaN,408,10 Barrel Brewing Company,Bend,OR
2,2265,Devil's Cup,American Pale Ale (APA),12.0 oz.,0.066,NaN,177,18th Street Brewery,Gary,IN
3,2264,Rise of the Phoenix,American IPA,12.0 ounce,0.071,NaN,177,18th Street Brewery,Gary,IN
4,2263,Sinister,American Double / Imperial IPA,12.0 oz,0.09%,NaN,177,18th Street Brewery,Gary,IN
5,2262,Sex and Candy,American IPA,12.0 OZ.,0.075,NaN,177,18th Street Brewery,Gary,IN


In [34]:
label = pd.read_csv('raha/datasets/beers/clean.csv')
label.set_index('index',inplace=True)
label.head()

,id,beer_name,style,ounces,abv,ibu,brewery_id,brewery_name,city,state
index,,,,,,,,,,
1,1436,Pub Beer,American Pale Lager,12.0,0.050,NaN,408,10 Barrel Brewing Company,Bend,OR
2,2265,Devil's Cup,American Pale Ale (APA),12.0,0.066,NaN,177,18th Street Brewery,Gary,IN
3,2264,Rise of the Phoenix,American IPA,12.0,0.071,NaN,177,18th Street Brewery,Gary,IN
4,2263,Sinister,American Double / Imperial IPA,12.0,0.090,NaN,177,18th Street Brewery,Gary,IN
5,2262,Sex and Candy,American IPA,12.0,0.075,NaN,177,18th Street Brewery,Gary,IN


In [35]:
def calculate_precision_recall(data_df, label_df, result_df):  
    # 确保DlocaFrame的索引一致  
    if data_df.index.nlevels != 1 or label_df.index.nlevels != 1 or result_df.index.nlevels != 1:  
        raise ValueError("DlocaFrames must have a single level index.")  
      
    # 处理NaN值  
    # data_df, label_df, result_df = data_df.replace([np.inf, -np.inf], np.nan), label_df.replace([np.inf, -np.inf], np.nan), result_df.replace([np.inf, -np.inf], np.nan)  
    # data_df, label_df, result_df = data_df.dropna(), label_df.dropna(), result_df.dropna()  
      
    # 初始化计数器  
    tp = 0  # True Positives  
    fp = 0  # False Positives  
    fn = 0  # False Neglocives  
    count,edit = 0,0
    missing_keys = set()
    # 遍历DataFrame，找出修改的数据点  
    # for index, (data_val, label_val, result_val) in zip(data_df.index, zip(data_df.values, label_df.values, result_df.values)): 
    #     # 比较修改后的数据点和标签  
    #     logging.info(f"index={index},data_val={data_val},label_val={label_val},result_val={result_val}")
    #     if data_val != result_val:  
    #         # 如果修改后的数据点与标签匹配，则增加TP计数  
    #         if result_val == label_val:  
    #             tp += 1  
    #         # 如果修改后的数据点与标签不匹配，则增加FP计数（这里修改了判断条件）  
    #         elif result_val != label_val and data_val == label_val:  # 仅在数据点与标签匹配时增加FP计数  
    #             fp += 1  
    #         # 如果原始数据点与标签不匹配，但修改后匹配，则增加FN计数  
    #         elif data_val != label_val and result_val == label_val:  
    #             fn += 1  
        # 遍历DataFrame，找出修改的数据点
    for index  in data_df.index : 
        for col in  data_df.columns :
            logging.info(f"index={index},col={col}")
            label = label_df.loc[index, col]
            data = data_df.loc[index, col]
            try:
                result = result_df.loc[index,col]
            except:
                # logging.warning(f"key {index} not found.")
                missing_keys.add(index)
                continue
            count += 1
            logging.debug(f"data={data},label={label},result={result}")
            # 比较修改后的数据点和标签
            if data != result:
                edit += 1
                # 如果修改后的数据点与标签匹配，则增加TP计数
                if result == label:
                    tp += 1
                # 如果修改后的数据点与标签不匹配，则增加FP计数
                if result != label:
                    fp += 1
                # 如果原始数据点与标签不匹配，但修改后匹配，则增加FN计数
            if data != label and result == data: # 需要改但没改的
                    fn += 1

  
    # 计算Precision和Recall  
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0  
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0  
    logging.warning(f"补全后的结果丢失数据{len(missing_keys)}条，dirty中有{len(data_df)}条数据，补全后为{len(result_df)}")
    logging.debug(f"丢失的数据分别是{missing_keys}")
    logging.info(f"tp={tp},fp={fp},fn={fn},count={count},edit={edit}")
    return precision, recall

In [36]:

ans = calculate_precision_recall(data, label, result)
ans


(0.15503875968992248, 0.021447721179624665)